# III) Poisson problem

We consider the following classical Poisson problem (inspired from electrostatics)

$$ 
\begin{array}{rcll}
-\nabla \cdot (\epsilon \nabla u) & = & \rho & \text{in}~\Omega \\
\text{some boundary conditions} &  & & \text{on} & \partial \Omega
\end{array}
$$


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

_____
## 1) Variational formulation

**Method** : 
1. Chose the appropriate function space
2. Multiply by a test function $v$ and integrate
3. Decrease the derivative order using Green formula to avoid derivative of discontinuous functions

**NB** : derivate discontinuous functions is possible using distribution theory, so what is before makes sense!

Let's consider $v\in H$ (we leave the definition of $H$ for later). Then

$$ - \int_\Omega v \nabla \cdot (\epsilon \nabla u) = \int_\Omega v \rho $$

- Leibniz formula : $\nabla \cdot (M \vec{A}) = M \nabla \cdot \vec{A} + \vec{A} \cdot \nabla M$
- Divergence theorem : $\int_\Omega \nabla \cdot \vec{A} = \int_{\partial \Omega} \vec{A} \cdot \vec{n}$

$$ \Rightarrow - \int_\Omega \nabla \cdot (v \epsilon \nabla u) + \int_\Omega \nabla v \cdot \epsilon \nabla u =  \int_\Omega v \rho$$
$$ \Rightarrow \int_\Omega \nabla v \cdot \epsilon \nabla u - \int_{\partial \Omega} v \epsilon \nabla u \cdot n =  \int_\Omega v \rho$$

There is now one boundary term. On a part of this boundary, we can have "natural" boundary conditions :
- $\epsilon\nabla u \cdot n = g $ : Neumann
- $\epsilon\nabla u \cdot n = g - r u $ : Robin
 
Or essential boundary conditions, meaning we have to change the definition of the function space $H$:
- $u = u_D$ : Dirichlet
- periodicity, anti-periodicity, etc.

The space should be $H^1$ + essential boundary conditions.

_____
## 2) Homogeneous Dirichlet & Neumann


$$ 
\left \{\begin{array}{rcll}
-\nabla \cdot (\epsilon \nabla u) & = & \rho & \text{in}~\Omega \\
\epsilon \nabla u \cdot n & =  & 0 & \text{on}~\Gamma_N ~\text{(left \& top)} \\
 u & =  & 0 & \text{on}~\Gamma_D ~\text{(bottom \& right)} \\
\end{array} \right.
$$

Dirichlet should appear in the space 

$$H = \{u\in H^1(\Omega), u = 0 ~\text{on}~\Gamma_D \} $$

In [2]:
from utils.myGeometries import square
Omega = square(maxh=0.1)    # generates the domain
Draw(Omega)
print(f"boundary names = {Omega.GetBoundaries()}")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

boundary names = ('bottom', 'right', 'top', 'left')


In [3]:
rho = 1.
fes = H1(Omega, order = 1, dirichlet  = "bottom|right") # top & left are homogeneous Newmann

In [4]:
rho = 1.
eps = 1.

In [5]:
u, v = fes.TnT()
bf = BilinearForm(fes)
bf += grad(v) * eps * grad(u) * dx
lf = LinearForm(fes)
lf += v * rho * dx

In [6]:
bf.Assemble()
lf.Assemble()

In [7]:
sol = GridFunction(fes)
sol.vec.data += bf.mat.Inverse(freedofs = fes.FreeDofs(), inverse = "sparsecholesky") * lf.vec
Draw(sol)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

Have a look at the $\vec{E}$ field :

In [10]:
Draw(- grad(sol), Omega, 
     vectors={"grid_size" : 20, "offset" : 0.5 },
     settings = { "Objects" : { "Wireframe" : False, "Surface" : False }}) 

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fals…

BaseWebGuiScene

___
### Exercise : 

To integrate on a boundary, we use `ds("name")` where `name` is the label of the boundary (without labels, all boundaries are considered).
Write the variational formulation and solve the following problem :

$$ 
\left \{\begin{array}{rcll}
-\nabla \cdot (\epsilon \nabla u) & = & \rho & \text{in}~\Omega \\
\epsilon \nabla u \cdot n & =  & 0 & \text{on}~\Gamma_{N1} ~\text{(left)} \\
\epsilon \nabla u \cdot n & =  & 1 & \text{on}~\Gamma_{N1} ~\text{(right)} \\
 u & =  & 0 & \text{on}~\Gamma_{D} ~\text{(bottom \& top)} \\
\end{array} \right.
$$